# Keyword expansion 

In this exercise we are going to use the keyword expansion technique propsoed in `Computer-Assisted Keyword and Document Set Discovery from Unstructured Text` by King, Lam and Roberts (2017), in order to label a dataset of tweets according to whether or not they are related to covid-19. 

The idea is to use an initial list of keywords to label the date, and then use supervised learning to expand the list of keywords to get a better sense of how people talk about a topic. It is an iterative approach, meaning that you start with a list of keywords, and expand it, run it again etc. until you saturate the list. The approach also emphasises that you should read some of the text that you label, in order to ensure correct labelling. 


#### NOTER
find words that are distinct to the documents that look like documents in the seed data.
In this we use logistic regression and loglikelyhood, but you can use any methods. (Related to slide 31, we use log regression in step 2, and log in step 3, but this is not the only way).

This exercise is a python translation of Gregory Eady's R exercise, heavily inspired by the replication material found here: https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/FMJDCD. If interested, you can also see Greg's walk-through of the R version of this code in his video here: https://gregoryeady.com/SocialMediaDataCourse/readings/Keywords/

### Read in required packages 

In [2]:
import pandas as pd
from nltk.stem.snowball import SnowballStemmer
import re
from tqdm import tqdm
from collections import OrderedDict
from collections import defaultdict
from collections import namedtuple
import numpy as np
from nltk.stem import PorterStemmer
from nltk.tokenize import WhitespaceTokenizer
import nltk
from sklearn.feature_extraction.text import CountVectorizer
import random
from math import lgamma
from sklearn import linear_model
import matplotlib.pyplot as plt
import datetime

# 1. Load the data

Read in data as usual. 

In [31]:
df = pd.read_csv("../dataset/MOC_Tweets.csv")

# 1.1. Preprocessing 

Due to time restraints, the preprocessing code is given below, ready to be run. Take a look at the code to understand what is being done. 


Subset the data by removing tweets before 2019 (we are only interested in tweets that may reference COVID-19).

In [32]:
df = df[df.date  >= 20190101] # Subset to 2019 and later because we'll look at COVID-19 over time
df = df.loc[df.tweet_id.drop_duplicates().index] # removing duplicate observations (tweets)

In [33]:
df.reset_index(inplace = True, drop = True) 

Save the original text and lowercase the text column.

In [34]:
df['text_original'] = df['text']
df['text'] = df['text'].str.lower()

Do some (but not all) preprocessing by removing tweet elements that we do not care about. 


In [35]:
# Remove mentions (posts that start with a "@some_user_name ")
df['text'] = df['text'].str.replace("\\B@\\w+|^@\\w+", "", regex = True)
# Change ampersands to "and"
df['text'] = df['text'].str.replace("&amp;", "and")
# Remove the "RT" and "via" (old retweet style)
df['text'] = df['text'].str.replace("(^RT|^via)((?:\\b\\W*@\\w+)+)","", regex=True, case=False)
# Remove URLs             
df['text'] = df['text'].str.replace("(https|http)?:\\/\\/(\\w|\\.|\\/|\\?|\\=|\\&|\\%)*\\b", "", regex = True)
# Keep ASCII only (removes Cyrillic, Japanese characters, etc.)
df['text'] = df['text'].str.replace("[^ -~]", "", regex = True)
# Remove double+ spaces (e.g. "build   the wall" to "build the wall")
df['text'] = df['text'].str.replace("\\s+", " ", regex = True)

With our mostly preprocessed tweets, let us begin building our classifier from chosen keywords. 

# 2. Define inclusion and exclusion keywords

You should now define the initial keywords that you want to include and exclude. Keywords to include should reference COVID-19, e.g. "covid19" and/or "coronavirus". We will use these initial keywords to find more keywords relevant to the topic.

1. Define 4 lists: the **first** should contain a seed reference word to be included, the **second** should contain the expanded list of reference words to include (empty to begin with), the **third** should contain a seed reference word to be excluded (can be left empty), and the **fourth** should contain the expanded list of reference words to exclude (empty to begin with). 

2. Using `.join`, collapse the two inclusion and exclusion lists, respectively, into strings that can be used as regex OR-operations. The result should be in the form \['dog', 'cat'\] --> 'dog|cat'

3. Use this regex string to create a bool column indicating whether the tweet contains one of your keywords.

4. If you have any exlusions, also find the tweets that contain the excluded keywords (the exclusion list can be left empty). 

5. Define a variable that is either 0 or 1, where 1 shows that the tweet contains one or more of your inclusion keywords _and_ does not contain any exclusion keywords. Create a bool column with this. 

6. See how many tweets you have labelled as related to COVID-19 so far (how many 0s and how many 1s). 

7. Sample 10 tweets labelled as COVID-19, and read the text in them (in the text_original column).

In [36]:
# Define the seed reference words to be included and excluded
included_seed_word = 'Covid-19'
excluded_seed_word = 'negative'

# Define the expanded list of reference words to be included and excluded
included_expanded_words = ['covidvirus', 'covid-19', 'covid', 'corona']
#excluded_expanded_words = [']

# Collapse the inclusion and exclusion lists into regex OR-operations
included_regex = '|'.join([included_seed_word] + included_expanded_words)
#excluded_regex = '|'.join([excluded_seed_word] + excluded_expanded_words)


# Print the four lists
print('Seed word to be included:', included_seed_word)
print('Expanded list of reference words to include:', included_expanded_words)
#print('Seed word to be excluded:', excluded_seed_word)
#print('Expanded list of reference words to exclude:', excluded_expanded_words)


Seed word to be included: Covid-19
Expanded list of reference words to include: ['covidvirus', 'covid-19', 'covid', 'corona']


In [37]:
# Laver funktion
def true_reg(reg, text):
    t = re.search(reg, text)
    if t != None:
        return(True)


# Laver regex udtryk (Lige nu dealer jeg ikke med mellemrum)
reg = re.compile(included_regex)

# forloop udført
m_liste = []
for x in df["text"]:
    if true_reg(reg, x)==True:
        m_liste.append(1)
    else:
        m_liste.append(0)

df["covid"] = m_liste

# Smartere måde at gøre det på: 
df["covid"] = [1 if re.search(reg, x) else 0 for x in df["text"]]

print("We have {} tweets about covid".format(len(df[df["covid"]==1])))
print("We have {} tweets not about covid".format(len(df[df["covid"]==0])))

We have 3215 tweets about covid
We have 576605 tweets not about covid


In [38]:
for x in df[df["covid"]==1].sample(10)["text_original"]:
    print(x)

In light of the spread of the coronavirus, each of us has a role to play to keep our communities healthy and safe.… https://t.co/3h5K09vQGJ
#coronavirus With this kind of impact in China, there will also be a disruptive impact on America’s economy - along… https://t.co/rsB65fUUft
Stay healthy &amp; safe during the #COVID19 outbreak.  🧼Wash hands for at least 20 seconds.   🛌If you feel unwell, STAY… https://t.co/0xoZaleD0C
Last year the US imported more than $13 billion of drugs, devices, &amp; food from China. With #coronavirus showing no… https://t.co/75FWTOkTqA
Read more about the 2019 Novel Coronavirus, how it spreads and how to protect you and your family on the CDC’s info… https://t.co/fdM7rdeyFk
The #coronavirus has claimed the lives of over 100 people in #China, &amp; there are thousands of confirmed cases acros… https://t.co/Gf34pc3jF6
Immigration status shouldn’t prevent a person from seeking care. Not receiving care during this coronavirus crisis… https://t.co/tqqw6HPljd
Dr. 

# 3. Further preprocessing and vectorizing

Next, we need to tokenize the data and preprocess the tokens (as opposed to the preprossesing on the full string as earlier). 

We will also remove all the keywords that demarcate exclusion and inclusion from the covid-19 theme. This is becasue we want the model to learn to predict the topic using other, new keywords. 

1. Create a new col named "text_preprocessed" - it should be equal the text col, but with the keywords removed (Hint: use `.str.replace()` with `regex = True`). 

----- 

To spend less time on lessons you have already been through, code for further preprocessing is provided. This code may take a few minutes to run. The steps are: 

2. Tokenizing. A whitespace tokenizer is used, since we want to keep words with '-'.

3. Removing any tokens that are only numbers (you can remove more types of tokens if you want - up to you).

4. Remove any empty strings.

5. Stemming.

6. Re-joining the stemmed tokens using a whitespace.

7. Creating a column with the preprocessed sentences.

----- 

8. Now you have a column  of sentences made out of stemmed and preprocessed tokens. Use a CountVectorizer to make a document term matrix based on this column. Set `min_df = 10` and `max_df = 0.999`, as well as `stop_words = 'english'` and set an appropriate `ngram_range`. 

NB: Do not try to make this DTM into a dataframe or np array, as you will most likely run out of memory. It is a sparse matrix that you can work with in the same way as an np.array.



In [39]:
#Create a new text column with both inclusion and exclusion keywords removed

df['text_preprocessed'] = df["text"].str.replace(reg,"", regex=True)
for x in df[df["covid"]==1].sample(10)["text_preprocessed"]:
    print(x)

great visit in south bend discussing healthcare and why indiana is well-prepared for the virus. 
house republicans are working with the to aggressively respond to the #19 threat. we must put th 
while there have been no confirmed virus cases in nevada, im glad to have had productive discussions with th 
a guide: how to prepare your home for virus 
house republicans press trump officials on plans to contain virus at border 
attended another virus briefing with mike pence and key public health officials to stay updated on the re 
with taxpayers footing the bill for #virus treatment and vaccine research, we must continue to ask questions, 
stay informed. the latest info on the virus. virus disease 2019 () via 
joined this morning to discuss the admins response to the #virus as well as my forthcoming l 
i'm concerned about this choice for the vice president to lead the #virus response. one of the largest outbre 


In [40]:
tokenizer = WhitespaceTokenizer()
ps = PorterStemmer()

preprocessed_sents =[]

for sent in tqdm(df['text_preprocessed']):
    words = tokenizer.tokenize(sent)
    words = [re.sub(r'\d+', '', word) for word in words] #removing tokens that are only digits 
    words = [x for x in words if x] #removing empty strings
    sent_stem = [ps.stem(word) for word in words]
    
    sent_done = " ".join(sent_stem)
    preprocessed_sents.append(sent_done)

df['text_stemmed'] = preprocessed_sents

100%|██████████| 579820/579820 [02:36<00:00, 3711.73it/s]


In [41]:
# Create a document term matrix here
vectorizer = CountVectorizer(min_df=10, max_df=0.999, stop_words='english', ngram_range=(1,2))

# Fit the vectorizer to the 'text' column and transform it to a document term matrix
doc_term_matrix = vectorizer.fit_transform(df['text_preprocessed'])


# 4. Sample training data and make predictions

Let us sample some tweets we will use to train our classifier. 

1) Define two lists of indices: One list containing the indices of the tweets in the reference set (those labelled as belonging to the covid-19 topic), and another list containing N sample of tweets not from the reference set (N should be either 2x the amount of tweets in the reference set or 50000, whichever is smaller).

2) You now have 2 lists of indices – use these to subset the Document Term Matrix (where each row represents a tweet, and each column a token) and the reference set column in the dataframe (the labels). Define a train DTM and  a train labels object. 

3) Fit a cross validated lasso regression, using the DTM subset as input (X) and the reference subset as labels (y). This means that we are trying to predict whether a tweet is in the reference set using the term frequencies. (Hint:  use sklearn's logistic reg with lasso penalty *'linear_model.LogisticRegressionCV(penalty="l1", ...'*). This may take some time (approx. 5 min, depending on the size of your train data).

4) Use the fitted model to make predictions on the full DTM, and create a column in the dataframe called `predicted_raw` based on this. (Remember that the rows in the DTM correspond to the rows in the dataframe).

5) The prediction outputs propabilities and not classes, so check the standard deviation of the predicion_raw column - this will check if we actually have some variance in the prediction. This is just a sanity check.

6) Set a threshold of 0.25, and assign 1 or 0 to a new column called `predicted`, depending on whether the probability in `predicted_raw` is >= the threshold. (Note: Keep the threshold low if you want more tweets to get into the target set).

7) Create a column called `set_var`. This variable should be == "Reference" if the observation is in the reference set (our original covid-19 labels), "Target" if it is _predicted_ to be a covid-19 related tweet (1) and "Not target" if it is _predicted_ not to be (0).

8) Create a crosstable of the prediciton and set_var, to see how you model does (hint: use use `pd.crosstab()`). Examine the crosstab - what do the different entries mean? 

In [17]:
######### 1
# Assuming that you have a dataframe df with a column 'label' indicating whether each tweet belongs to the covid-19 topic
# Get the indices of the tweets in the reference set (i.e., those labelled as belonging to the covid-19 topic)
ref_indices = np.where(df['covid'] == 1)[0].tolist()

# Get N random indices of tweets not in the reference set
nonref_indices = np.where(df['covid'] != 1)[0]
# Set N to be either 2 times the number of tweets in the reference set or 50000, whichever is smaller
N = min(2 * len(ref_indices), 50_000)

nonref_indices = np.random.choice(nonref_indices, size=N, replace=False).tolist()

In [77]:
##### 2
indices = nonref_indices+ref_indices
DTM_subset = doc_term_matrix[indices] # Subset af alle tweets som går igen 
DTM_if_covid = df.iloc[indices]["covid"] # Liste over hvorvidt der er covidord i den eller ej

##### 3
from sklearn.linear_model import LogisticRegressionCV

clf = LogisticRegressionCV(cv=5, random_state=0).fit(DTM_subset, DTM_if_covid)

In [78]:
##### 4
df["predicted_raw"]= clf.predict(doc_term_matrix)

In [84]:
##### 5 (Standards deviations gav mere mening før i tiden, nu hvor vi faktisk får classes og ikke probabilities er det her bedre)
print ("Mean of A:", np.mean(df["covid"]))
print ("Mean of B:", np.mean(df["predicted_raw"]))

Mean of A: 0.005544824255803525
Mean of B: 0.012907454037459902


In [98]:
##### 6 (Giver ikke rigtig mening når vi nu har logistisk ting))


##### 7 ()
def set_new_col(row):
    if row["covid"]==1:
        return 'Reference'
    elif row['predicted_raw']==1:
        return 'Target'
    else:
        return 'Not Target'

df["set_var"]= df.apply(lambda row: set_new_col(row), axis=1)

##### 8 
pd.crosstab(df["set_var"], df[""])

predicted_raw,0,1
set_var,,
Not Target,572335,0
Reference,1,3214
Target,0,4270


# 5. Calculate the log likelihood as in the paper

1) Create 3 sets of indices based on the `set_var` colum: one for "Target", one for "Not target" and one for "Reference". 

2) Create 3 objects for the target, not_target and reference sets, based on the DTM. These should be: for each token, how often is the given token in the set, how many documents in the set contains the given token, and the proportion of documents in the set containing the given token. (Hint: see sample code for the target set. If you want to convert to a list and not a matrix object, you can use the `.tolist()[0]`)

3) Create a new dataframe, where each row is a token from the DTM (you can use `vectorizer.get_feature_names()`), with 9 cols for each of the 9 objects you just created. 

4) Subset the dataset by removing any observations where the terms do not appear in either the target or not_target set, thus keeping only tokens that were in the original search set (step (a) on page 979).

5) Keywords go in the target list if their proportion is higher among those documents estimated to be in the reference set than not; e.g. if for the word "pandemic", 15% of documents predicted as target contain the word "pandemic" versus only 2% among those in the not_target set (step (b) on page 979). Therefore: create a new column that should be True if the token has a higher or equal proportion in the target set than in the not_target set. 

6) Examine the `llik` function provide and look in the paper - what does it do? 

7) Calculate the amount of documents in the target and the not_target set.

8) Use the provided function to calculate the log likelihood for each token. Assign this to a new column in the dataframe created in step 3. 


In [134]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer.get_feature_names_out()

array(['00', '00 00', '00 00pm', ..., 'zuckerberg', 'zuckerberg facebook',
       'zuckerberg testifies'], dtype=object)

In [136]:
## 2
# create objects for target set
target_counts = doc_term_matrix[target_ids].sum(axis=0).tolist()[0]
target_doc_counts = doc_term_matrix[target_ids].sign().sum(axis=0).tolist()[0]
target_doc_proportions = (doc_term_matrix[target_ids] > 0).mean(axis=0).tolist()[0]

# create objects for not_target set
not_target_counts = doc_term_matrix[not_target_ids].sum(axis=0).tolist()[0]
not_target_doc_counts = doc_term_matrix[not_target_ids].sign().sum(axis=0).tolist()[0]
not_target_doc_proportions = (doc_term_matrix[not_target_ids] > 0).mean(axis=0).tolist()[0]

# create objects for reference set
reference_counts = doc_term_matrix[ref_ids].sum(axis=0).tolist()[0]
reference_doc_counts = doc_term_matrix[ref_ids].sign().sum(axis=0).tolist()[0]
reference_doc_proportions = (doc_term_matrix[ref_ids] > 0).mean(axis=0).tolist()[0] 


## Giant dataset
df_test = pd.DataFrame(list(zip(target_counts, target_doc_counts, target_doc_proportions, not_target_counts, not_target_doc_counts, not_target_doc_proportions, reference_counts, reference_doc_counts, reference_doc_proportions)),
                       index=vectorizer.get_feature_names_out(), 
                  columns=['target_counts', 'target_doc_counts', 'target_doc_proportions', 'not_target_counts', 'not_target_doc_counts', 'not_target_doc_proportions', 'reference_counts', 'reference_doc_counts', 'reference_doc_proportions'])

In [137]:
df_test

,target_counts,target_doc_counts,target_doc_proportions,not_target_counts,not_target_doc_counts,not_target_doc_proportions,reference_counts,reference_doc_counts,reference_doc_proportions
00,26,20,0.004684,1648,1320,0.002306,4,4,0.001244
00 00,2,2,0.000468,140,130,0.000227,0,0,0.000000
00 00pm,1,1,0.000234,11,10,0.000017,0,0,0.000000
00 10,0,0,0.000000,13,13,0.000023,0,0,0.000000
00 11,2,2,0.000468,24,24,0.000042,1,1,0.000311
...,...,...,...,...,...,...,...,...,...
zotzotzot,0,0,0.000000,11,11,0.000019,0,0,0.000000
zte,0,0,0.000000,41,40,0.000070,0,0,0.000000
zuckerberg,3,3,0.000703,175,168,0.000294,0,0,0.000000
zuckerberg facebook,0,0,0.000000,10,10,0.000017,0,0,0.000000


In [132]:
df_test

,target_counts,target_doc_counts,target_doc_proportions,not_target_counts,not_target_doc_counts,not_target_doc_proportions,reference_counts,reference_doc_counts,reference_doc_proportions
0,26,20,0.004684,1648,1320,0.002306,4,4,0.001244
1,2,2,0.000468,140,130,0.000227,0,0,0.000000
2,1,1,0.000234,11,10,0.000017,0,0,0.000000
3,0,0,0.000000,13,13,0.000023,0,0,0.000000
4,2,2,0.000468,24,24,0.000042,1,1,0.000311
...,...,...,...,...,...,...,...,...,...
124708,0,0,0.000000,11,11,0.000019,0,0,0.000000
124709,0,0,0.000000,41,40,0.000070,0,0,0.000000
124710,3,3,0.000703,175,168,0.000294,0,0,0.000000
124711,0,0,0.000000,10,10,0.000017,0,0,0.000000


In [91]:
# Equation from King et al. (2017) p. 979 (using logs for stability) to calculate the likelihood

def llik(target_num_docs, nottarget_num_docs, target_num_docs_total, nottarget_num_docs_total):
    
    '''No docstring - you neew to see what it does :) '''
    
    x1 = ((lgamma(target_num_docs + 1) + lgamma(nottarget_num_docs + 1)) -
           lgamma(target_num_docs + nottarget_num_docs + 1 + 1))
    
    x2 = ((lgamma(target_num_docs_total - target_num_docs + 1) +
           lgamma(nottarget_num_docs_total - nottarget_num_docs + 1)) -
           lgamma(target_num_docs_total - target_num_docs +
           nottarget_num_docs_total - nottarget_num_docs + 1 + 1))
    
    llik = x1 + x2
    
    return llik


# 6. Examine new keywords

1) Show the top 25 keywords based on highest log likelihood, where the share of documents in the target set is higher than in the not_target set (see task 5.5). These are the tokens that are most likely to differentiate between the target and not_target sets (meaning that they help the model predict covid-19 related tweets).

2) Do the same with the not_target - what are these terms representative of? 

3) Are there any of these tokens that you want to include in the keywords? Choose 1-3 keywords that you want to include or exclude. 

4) For the 1-3 keywords you have found, find tweets that contain the given keyword in the original tweet text in the original dataframe. Read some tweets where the keyword is used in context - do you still want to include or exclude the keyword? 

5) Optional: add the new keywords to the original list at the beginning of this exercise in 2.1, and rerun the exercises until here, now including the new keywords. This is how the computer-assisted keyword discovery is used iteratively. 


# 7. Optional: Use your new classifier for downstream tasks

1) Assign a `final_classification` boolean column in the original dataframe, which should be 1 if the tweet contains any of the keywords in the new, complete list and if it does not contain any of the exclusion keywords. 

2) Examine the value counts of the political affiliation variable. Assign "Democrat" to the tweets labelled with "Independent" (see the people behind the tweets for reason).

3) Plot the share of tweets labelled as covid-19 relevant by your classifier (y), grouped on days (x) for each party - meaning two lines of covid-19 share across time. 

**Hints:** <br>
The pandas `groupby` functionality may be of help to you. <br>
You can also also turn the date ints into so-called datetime objects using this:

`dates =[datetime.datetime(year=int(x[i][0:4]), month=int(x[i][4:6]), day=int(x[i][6:8])) for i in range(len(x))]`

where x is a list of the unique dates as int. 
